In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import os
import cv2
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from transformers import pipeline
from transformers import AutoModelForSequenceClassification
import torch
from transformers import TFAutoModel, AutoTokenizer
from datasets import Dataset
from torch.utils.data import DataLoader

In [4]:
# 데이터셋 경로 설정
data_path = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/data/'  # 이미지 및 텍스트 파일 폴더
# 텍스트 전처리 함수
def preprocess_text(text):
    text = text.lower()  # 소문자로 변환
    text = ''.join(e for e in text if e.isalnum() or e.isspace())  # 특수문자 제거
    return text

# 이미지 전처리 함수
def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    if image == None:
      print(1)
    image = cv2.resize(image, target_size)  # 크기 조정
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # RGB로 변환
    image = image / 255.0  # 정규화
    return image

# 데이터 로드 함수
def load_data(folder_path, target_size=(224, 224)):
    texts, images = [], []

    # 폴더 내 파일 목록 가져오기
    file_names = [f.split('.')[0] for f in os.listdir(folder_path) if f.endswith('.jpg')]  # .jpg 파일 기준
    unique_ids = set(file_names)  # 중복 제거

    for file_id in unique_ids:
        # 이미지와 텍스트 파일 경로 생성
        image_file = os.path.join(folder_path, f"{file_id}.jpg")
        text_file = os.path.join(folder_path, f"{file_id}.txt")

        # 이미지와 텍스트 파일이 모두 있는 경우 로드
        if os.path.exists(image_file) and os.path.exists(text_file):
            # 이미지 로드 및 전처리
            image = preprocess_image(image_file, target_size)
            images.append(image)

            # 텍스트 로드 및 전처리
            with open(text_file, 'r') as file:
                raw_text = file.read().strip()
            text = preprocess_text(raw_text)
            texts.append(text)

    return texts, images

In [5]:
texts, images = [], []

# 폴더 내 파일 목록 가져오기
file_names = [f.split('.')[0] for f in os.listdir(data_path) if f.endswith('.jpg')]  # .jpg 파일 기준
unique_ids = set(file_names)  # 중복 제거

In [7]:
count = 0
for file_id in unique_ids:
    # 이미지와 텍스트 파일 경로 생성
    image_file = os.path.join(data_path, f"{file_id}.jpg")
    text_file = os.path.join(data_path, f"{file_id}.txt")

    # 이미지와 텍스트 파일이 모두 있는 경우 로드
    if os.path.exists(image_file) and os.path.exists(text_file):
        # 이미지 로드 및 전처리
        image = cv2.imread(image_file)
        if str(type(image)) == "<class 'NoneType'>":
          continue
        image = cv2.resize(image, (224, 224))  # 크기 조정
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # RGB로 변환
        image = image / 255.0  # 정규화
        images.append(image)

        # 텍스트 로드 및 전처리
        with open(text_file, 'r') as file:
            raw_text = file.read().strip()
        text = preprocess_text(raw_text)
        texts.append(text)

In [8]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [9]:
# 1. Hugging Face Dataset 생성
# 이미지와 텍스트 데이터를 함께 사용
data_dict = {
    "text": texts,  # 텍스트 데이터
    "image": images  # 이미지 데이터 (예시)
}
dataset = Dataset.from_dict(data_dict)


OverflowError: There was an overflow with type <class 'list'>. Try to reduce writer_batch_size to have batches smaller than 2GB.
(offset overflow while concatenating arrays)

In [ ]:
save_path = '/content/drive/MyDrive/Colab Notebooks/COSE474/MVSA/dataset'
dataset.save_to_disk(save_path)

In [ ]:
from datasets import load_from_disk

# 저장된 데이터셋 로드
loaded_dataset = load_from_disk(save_path)
print(loaded_dataset)

In [ ]:

# 2. 데이터셋 전처리 (토크나이저 사용)
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

# 텍스트 토크나이즈
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# PyTorch DataLoader 생성
def collate_fn(batch):
    # 텍스트 입력과 이미지를 함께 반환
    input_ids = torch.stack([b["input_ids"][0] for b in batch]).to(device)
    attention_mask = torch.stack([b["attention_mask"][0] for b in batch]).to(device)
    images = torch.tensor([b["image"] for b in batch]).to(device)
    return {"input_ids": input_ids, "attention_mask": attention_mask}, images

# DataLoader 설정
batch_size = 32
data_loader = DataLoader(tokenized_dataset, batch_size=batch_size, collate_fn=collate_fn)


In [ ]:

# 3. 모델 예측 (배치 처리)
all_labels = []
all_probabilities = []

for batch in data_loader:
    inputs, _ = batch  # 텍스트 입력만 사용
    with torch.no_grad():
        outputs = model(**inputs)

    # 소프트맥스 확률 계산
    probabilities = F.softmax(outputs.logits, dim=1)
    labels = torch.argmax(probabilities, dim=1).cpu().numpy()

    # 결과 저장
    all_labels.extend(labels)
    all_probabilities.extend(probabilities.cpu().numpy())

In [ ]:
# 4. 결과 매핑 및 출력
label_map = {0: "negative", 1: "positive"}
all_labels = [label_map[label] for label in all_labels]

# 첫 번째 배치 결과 출력
for text, label, prob in zip(texts[:batch_size], all_labels[:batch_size], all_probabilities[:batch_size]):
    print(f"Text: {text}")
    print(f"Predicted Sentiment: {label}")
    print(f"Probabilities: {prob}")

> 여기까지 1차 task: 전처리+추후 학습에 쓸 이미지 pos/neg 비율(이건 러프하게 잡아도 될 것 같음. 어차피 학습을 서로 돌리면서 맞출 거니까)

In [2]:

# ALBert 모델과 토크나이저 로드
albert_model_name = "albert-base-v2"  # Hugging Face 모델 이름
tokenizer = AutoTokenizer.from_pretrained(albert_model_name)
albert_model = TFAutoModel.from_pretrained(albert_model_name)

# 입력 텍스트 처리 함수
def preprocess_texts(texts, max_length=128):
    tokenized = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )
    # 'token_type_ids' 제거
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"]
    }

# BiLSTM 모델 정의
def build_bilstm_model(albert_model, lstm_units=128):
    # ALBert 입력
    input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

    # ALBert 임베딩 추출
    albert_outputs = albert_model(input_ids, attention_mask=attention_mask)
    albert_embeddings = albert_outputs.last_hidden_state  # (batch_size, seq_length, hidden_size)

    # BiLSTM 적용
    bilstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(lstm_units, return_sequences=False)
    )(albert_embeddings)

    # 최종 출력
    output = tf.keras.layers.Dense(lstm_units, activation="relu")(bilstm)

    # 모델 정의
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
    return model

# 텍스트 예제
texts = [
    "This is a positive example.",
    "I feel sad and lonely."
]

# 텍스트 전처리
tokenized_inputs = preprocess_texts(texts)

# BiLSTM 모델 빌드
bilstm_model = build_bilstm_model(albert_model)

# 모델 출력 확인
features = bilstm_model(tokenized_inputs)
print(features.shape)  # (batch_size, lstm_units)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['predictions.LayerNorm.weight', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing TFAlbertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


(2, 128)
